In [6]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adamax, Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical
from copy import deepcopy

In [7]:
import import_ipynb

import Model 
from Model import *

import utils
from utils import *

In [8]:
data = pd.read_pickle('mon_data.pkl')

In [9]:
from tensorflow.keras.models import load_model

# 모델 로드
model = load_model('nonbase_model.h5')

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)  # 즉시 실행 모드 강제

# first part 불러오기
first_part = split_by_label(data, 0, 49)
train_f, test_f = split_train_test(first_part, test_size=0.2, random_state=11)

# second part 불러오기
second_part = split_by_label(data, 50, 94)
train, test = split_train_test(second_part, test_size=0.2, random_state=11)

# test 데이터셋 합치기
X_test_combined = np.concatenate([
    test_f['Direction_Sequence'].values,
    test['Direction_Sequence'].values
], axis=0)

y_test_combined = np.concatenate([
    test_f['Label'].values,
    test['Label'].values
], axis=0)

combined_test = pd.concat([test_f, test], axis=0).reset_index(drop=True)

# 3D ndarray 로 변환 
train_seq, train_label = split_data_label(train)

from sklearn.preprocessing import LabelEncoder

# 레이블을 0~44 범위로 재인덱싱
le = LabelEncoder()
train_label = le.fit_transform(train_label)

train_seq = np.stack(train_seq.values)
train_seq = train_seq[..., np.newaxis]

# train_label = train_label.values
train_label = to_categorical(train_label, num_classes=95)

In [19]:
# 1.1 데이터 형태 확인
print(f"combined_test shape: {combined_test.shape}")  # (원본 test_f + test 행 개수, 2)여야 함
print(f"컬럼 목록: {combined_test.columns.tolist()}")  # ['Direction_Sequence', 'Label'] 확인

# 1.2 라벀 범위 검증
print("라벨 최소/최대:", combined_test['Label'].min(), combined_test['Label'].max())  
# 0~94 범위여야 함 (데이터 분할 로직에 따라 0~49 + 50~94)

combined_test shape: (3800, 2)
컬럼 목록: ['Direction_Sequence', 'Label']
라벨 최소/최대: 0 94


In [20]:
# 모델 훈련
history = model.fit(x=train_seq, y=train_label, epochs=30, verbose=1)
print(f"Data_B training accuracy: {history.history['accuracy'][-1]:.4f}")

Epoch 1/30


c:\Users\swu\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


225/225 [==============================] - 21s 92ms/step - loss: 0.0462 - accuracy: 0.9871
Epoch 2/30
225/225 [==============================] - 22s 97ms/step - loss: 0.0518 - accuracy: 0.9860
Epoch 3/30
225/225 [==============================] - 22s 98ms/step - loss: 0.0501 - accuracy: 0.9864
Epoch 4/30
225/225 [==============================] - 21s 92ms/step - loss: 0.0462 - accuracy: 0.9876
Epoch 5/30
225/225 [==============================] - 20s 89ms/step - loss: 0.0427 - accuracy: 0.9875
Epoch 6/30
225/225 [==============================] - 21s 94ms/step - loss: 0.0420 - accuracy: 0.9871
Epoch 7/30
225/225 [==============================] - 22s 100ms/step - loss: 0.0424 - accuracy: 0.9881
Epoch 8/30
225/225 [==============================] - 22s 96ms/step - loss: 0.0401 - accuracy: 0.9883
Epoch 9/30
225/225 [==============================] - 22s 98ms/step - loss: 0.0360 - accuracy: 0.9883
Epoch 10/30
225/225 [==============================] - 21s 92ms/step - loss: 0.0358 - accura

### 95개의 클래스로 model test

In [21]:
test_seq, test_label = split_data_label(combined_test) # test + first part에서 불러온 데이터
test_seq = np.stack(test_seq.values)
test_seq = test_seq[..., np.newaxis]

test_label = test_label.values
test_label = to_categorical(test_label, num_classes=95)

test = tf.data.Dataset.from_tensor_slices((test_seq, test_label))
test = test.batch(32) 

final_accuracy = model.evaluate(test)
print(final_accuracy)


119/119 [==============================] - 8s 68ms/step - loss: 24.4396 - accuracy: 0.0103
[24.439640045166016, 0.01026315800845623]
